### Canned laughter removal (and optionally, generation)
We will use the model(s) developed in the other notebook that are able to tell laughter apart from everything else
to detect laughter in an episode and rewrite it with something else

In [ ]:
import sys
sys.path.append('../utils/')

In [ ]:
# local imports
import utils
import episode
import color
import stats
import modelbuilder
# stdlib imports
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy.io import wavfile
from pathlib import Path
from matplotlib import pyplot as plt
import pyaudio
import pydub

In [ ]:
weightspath = 'task:per-season-split-ckpt.hdf5'

In [ ]:
model = modelbuilder.build_laugh_model()
model.load_weights(weightspath)
model = modelbuilder._compile_binary(model)

In [ ]:
archived_episodes = ['friends-s02-e{:0>2d}'.format(i) for i in range(1, 5)] + ['friends-s03-e{:0>2d}'.format(9)]

In [ ]:
for ep in archived_episodes:
    wavaccum = defaultdict(list)
    
    decoded, preds = episode.detect_in_episode(ep, model, algorithms=['hmm'], precision=2)
    sr, wavdata = wavfile.read('../wav/{}.wav'.format(ep))
    patches = episode.discrete_to_chunks(decoded['hmm'])
    
    precision = 2
    mult = .96 / precision * sr
    for label in range(2):
        for s, e in patches[label]:
            start_f, end_f =  int(mult * s), int(mult * e)
            this_chunk = wavdata[start_f:end_f]
            wavaccum[label] += this_chunk.tolist()
            
    for label in range(2):
        wavaccum[label] = np.array(wavaccum[label], dtype=wavdata.dtype)

    wavfile.write(rate=sr, data=wavaccum[0], filename='../data/archive/{}_non-laugh-concat.wav'.format(ep))
    wavfile.write(rate=sr, data=wavaccum[1], filename='../data/archive/{}_laugh-concat.wav'.format(ep))

In [ ]:
raise NotImplementedError

In [ ]:
p = pyaudio.PyAudio()
stream = p.open(format = pyaudio.paInt16,
                channels = 1,  
                rate = sr,  
                output = True)

In [ ]:
aud = wavaccum[1][sr*160:sr*170]
stream.write(aud)

In [ ]:
raise NotImplementedError(NotImplemented)

In [ ]:
for ep in archived_episodes:
    decoded, preds = episode.detect_in_episode(ep, model, algorithms=['viterbi'], precision=2)
    